In [25]:
from __future__ import annotations
import os
import sys

dir_current = os.path.dirname(os.path.abspath("__file__"))
dir_parent = os.path.dirname(dir_current)
dir_root = os.path.dirname(dir_parent)
dir_suite2p = os.path.join(dir_parent, "suite2p")
dir_optic = os.path.join(dir_root, "optic")
sys.path.insert(0, dir_suite2p)
sys.path.insert(0, dir_optic)

from PyQt5.QtWidgets import *
import json

from suite2p import run_s2p, default_ops

# opticモジュールを再インポート

from optic.type_definitions import *
from optic.config import *
from optic.manager import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.gui.bind_func import *

class Suite2pAutoRun(QMainWindow):
    def __init__(self):
        QMainWindow.__init__(self)
        self.widget_manager = initManagers(WidgetManager())
        self.path_config = os.path.join(dir_parent, "config", "suite2p_params.json")
        self.path_config_ops = os.path.join(dir_parent, "config", "ops_params.json")

        self.setupUI()
        self.chooseDatabaseDirectory()

    def setupUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.setGeometry(100, 100, 1900, 1000)
        self.layout_main = QGridLayout(self.central_widget)
        self.layout_main.addLayout(self.makeLayoutMain(), 0, 0)

        self.bindFuncAllWidget()

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    def makeLayoutComponentSuite2pParameters(self):
        layout = QGridLayout()

        suite2p_params = json.load(open(self.path_config, "r"))["suite2p_parameters"]
        for key_settings_col in suite2p_params.keys():
            grid_position = suite2p_params[key_settings_col]["grid_position"]
            dict_settings = suite2p_params[key_settings_col]["parameters"]
            self.widget_manager.makeWidgetLabel(
                key=key_settings_col, 
                label=key_settings_col, 
                align=Qt.AlignCenter, 
                font_size=16, 
                bold=True,
                )
            layout.addWidget(self.widget_manager.dict_label[key_settings_col], grid_position[0], grid_position[1])
            for key_setting in dict_settings.keys():
                grid_position = dict_settings[key_setting]["grid_position"]
                value_default = dict_settings[key_setting]["value_default"]
                if key_setting == "nplanes" or key_setting == "fs":
                    color = "red"
                else:
                    color = "black"
                layout_ = makeLayoutLineEditLabel(
                    widget_manager=self.widget_manager, 
                    key_label=f"param_{key_setting}", 
                    key_lineedit=f"param_{key_setting}", 
                    label=key_setting,
                    text_set=str(value_default),
                    color=color
                    )
                layout.addLayout(layout_, grid_position[0], grid_position[1])

        return layout
    
    def makeLayoutComponentOpsFile(self):
        layout = QVBoxLayout()
        layout_top = QHBoxLayout()
        layout_bottom = QHBoxLayout()
        layout_top.addWidget(self.widget_manager.makeWidgetLabel(key="ops_file", label="Ops config file"))
        layout_bottom.addWidget(self.widget_manager.makeWidgetComboBox(key="dir_database_sub"))
        layout_bottom.addWidget(self.widget_manager.makeWidgetButton(key="ops_save", label="save ops"))
        layout_bottom.addWidget(self.widget_manager.makeWidgetButton(key="ops_load", label="load ops"))
        layout_bottom.addWidget(self.widget_manager.makeWidgetButton(key="ops_delete", label="delete ops"))
        layout.addLayout(layout_top)
        layout.addLayout(layout_bottom)
        return layout
    
    def makeLayoutComponentIOListWidget(self):
        layout = QHBoxLayout()
        layout_input = QVBoxLayout()
        layout_input.addWidget(self.widget_manager.makeWidgetLabel(key="input", label="Input TIFF/ND2 Directory", align=Qt.AlignCenter, font_size=16, bold=True))
        layout_input.addWidget(self.widget_manager.makeWidgetListWidget(key="input"))
        layout_output = QVBoxLayout()
        layout_output.addWidget(self.widget_manager.makeWidgetLabel(key="output", label="Output Mat File", align=Qt.AlignCenter, font_size=16, bold=True))
        layout_output.addWidget(self.widget_manager.makeWidgetListWidget(key="output", editable=True))
        layout.addLayout(layout_input)
        layout.addLayout(layout_output)
        layout.addLayout(self.makeLayoutComponentUpdateOutputPath())
        return layout
                         
    def makeLayoutComponentUpdateOutputPath(self):
        layout = QVBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetLabel(key="dir_database_sub", label="Database Subdirectory"))
        layout.addWidget(self.widget_manager.makeWidgetComboBox(key="dir_database_sub"))
        layout.addWidget(self.widget_manager.makeWidgetButton(key="update_output_path", label="Update"))
        return layout
    
    def makeLayoutComponentIOButton(self):
        layout = QHBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetButton(key="browse", label="browse"))
        layout.addWidget(self.widget_manager.makeWidgetButton(key="delete", label="delete"))
        layout.addWidget(self.widget_manager.makeWidgetButton(key="clear", label="clear"))
        return layout
    
    def makeLayoutComponentCheckboxLineEdit(self):
        layout = QHBoxLayout()
        layout.addLayout(makeLayoutLineEditLabel(
            widget_manager=self.widget_manager, 
            key_label="browse_option", 
            key_lineedit="browse_option", 
            label="browse including the following text, ex) nd2, 241101")
            )
        layout.addWidget(self.widget_manager.makeWidgetCheckBox(key="dir_recursive", label="Get directories recursively", checked=False))
        layout.addWidget(self.widget_manager.makeWidgetCheckBox(key="skip", label="Skip analyzed directories", checked=True))
        layout.addWidget(self.widget_manager.makeWidgetCheckBox(key="copy_database", label="Copy Fall.mat to database", checked=True))
        return layout
    
    def makeLayoutComponentRunExitButton(self):
        layout = QHBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetButton(key="run_suite2p", label="Run Suite2p"))
        layout.addWidget(self.widget_manager.makeWidgetButton(key="run_registration", label="Run Registration"))
        layout.addWidget(self.widget_manager.makeWidgetButton(key="exit", label="Exit"))
        return layout


    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    def makeFrameTop(self):
        frame = QFrame()
        frame.setFrameStyle(QFrame.Box | QFrame.Raised)
        frame.setLineWidth(2)

        layout = QVBoxLayout(frame)
        layout.addLayout(self.makeLayoutComponentSuite2pParameters())
        layout.addLayout(self.makeLayoutComponentOpsFile())
        return frame

    def makeFrameBottom(self):
        frame = QFrame()
        frame.setFrameStyle(QFrame.Box | QFrame.Raised)
        frame.setLineWidth(2)
        
        layout = QVBoxLayout(frame)
        layout.addLayout(self.makeLayoutComponentIOListWidget())
        layout.addLayout(self.makeLayoutComponentIOButton())
        layout.addLayout(self.makeLayoutComponentCheckboxLineEdit())
        layout.addLayout(self.makeLayoutComponentRunExitButton())
        return frame
    
    def makeLayoutMain(self):
        layout = QVBoxLayout()
        layout.addWidget(self.makeFrameTop())
        layout.addWidget(self.makeFrameBottom())
        return layout
    
    """
    Functions
    """
    def chooseDatabaseDirectory(self):
        self.dir_database = openFolderDialog(self, "Select database directory")
            
    def chooseInputDirectory(self):
        dir_input = openFolderDialog(self, "Select database directory")
        if self.widget_manager.dict_checkbox["dir_recursive"].isChecked():
            depth = None
        else:
            depth = 0
        dirs_input = getAllSubDirectories(dir_input, depth=depth)

        list_text_include = self.widget_manager.dict_lineedit["browse_option"].text().strip().split(",")
        dirs_input = getMatchedPaths(dirs_input, list_text_include)
        
        for dir_input in dirs_input:
            # skip if already exists
            if not dir_input in getAllItemsFromListWidget(self.widget_manager.dict_listwidget["input"]):
                self.widget_manager.dict_listwidget["input"].addItem(dir_input)

    def removePathsFromListWidget(self):
        removeSelectedItemFromListWidget(self.widget_manager.dict_listwidget["input"])
        removeSelectedItemFromListWidget(self.widget_manager.dict_listwidget["input"])

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncAllWidget(self):
        self.widget_manager.dict_button["browse"].clicked.connect(self.chooseInputDirectory)
        self.widget_manager.dict_button["remove"].clicked.connect(lambda: )
        self.widget_manager.dict_button["clear"].clicked.connect(lambda: clearListWidget)
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    applyAppStyle(app, font_name="Arial")
    gui = Suite2pAutoRun()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\suite2p\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
gui.widget_manager.dict_lineedit

{'param_nplanes': <PyQt5.QtWidgets.QLineEdit at 0x25a17a711f0>,
 'param_nchannels': <PyQt5.QtWidgets.QLineEdit at 0x25a17a713a0>,
 'param_functional_chan': <PyQt5.QtWidgets.QLineEdit at 0x25a17a71550>,
 'param_tau': <PyQt5.QtWidgets.QLineEdit at 0x25a17a71700>,
 'param_fs': <PyQt5.QtWidgets.QLineEdit at 0x25a17a718b0>,
 'param_do_bidiphase': <PyQt5.QtWidgets.QLineEdit at 0x25a17a71a60>,
 'param_bidiphase': <PyQt5.QtWidgets.QLineEdit at 0x25a17a71c10>,
 'param_multiplane_parallel': <PyQt5.QtWidgets.QLineEdit at 0x25a17a71dc0>,
 'param_ignore_flyback': <PyQt5.QtWidgets.QLineEdit at 0x25a17a71f70>,
 'param_preclassify': <PyQt5.QtWidgets.QLineEdit at 0x25a17a721f0>,
 'param_save_mat': <PyQt5.QtWidgets.QLineEdit at 0x25a17a723a0>,
 'param_save_NWB': <PyQt5.QtWidgets.QLineEdit at 0x25a17a72550>,
 'param_combined': <PyQt5.QtWidgets.QLineEdit at 0x25a17a72700>,
 'param_reg_tif': <PyQt5.QtWidgets.QLineEdit at 0x25a17a728b0>,
 'param_reg_tif_chan2': <PyQt5.QtWidgets.QLineEdit at 0x25a17a72a60>,
